In [1]:
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import pandas as pd
from bs4 import BeautifulSoup
import json
from IPython.display import clear_output

url = url = "http://academico.unemat.br/"


def site_login(url):

    option = Options()
    option.headless = True
    driver = webdriver.Firefox(options=option)

    login = str(input("Digite seu login: "))
    password = str(input("Digite sua senha: "))    
    
    driver.get(url)
    print("Waiting for connection: ", end="")
    driver.find_element_by_id("uid").send_keys(login)
    driver.find_element_by_id("pwd").send_keys(password)
    driver.find_element_by_id("btnLogin").click()
    driver.find_element_by_id("m19").click()
    driver.find_element_by_id("m19").click()
    element = driver.find_element_by_id("tbodym46")
    html_content = element.get_attribute("outerHTML")

    # Parse HTML (Parsear o conteúdo HTML) - BeaultifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find(name='table')

    # Data Structure Conversion (Estruturar conteúdo em um Data Frame) - Pandas
    df_full = pd.read_html(str(table))[0]

    driver.quit()

    return(df_full)


def process_df(df):
    cr, dem = 0, 1
    df = df.drop(columns=[1])
    df.index = df.index.astype(str, copy=False)
    df = df.drop("0")
    df = df.rename(columns={
        0: "Período",
        2: "Disciplina",
        3: "Professor",
        4: "Carga Horária",
        5: "Nota",
        6: "Frequência",
        7: "Estado",
        8: "Créditos"})
    for index, row in df.iterrows():
        df["Disciplina"][index] = df["Disciplina"][index].split("(")[0]
        df["Professor"][index] = " ".join(
            df["Professor"][index].split(" ")[3:])
        df["Créditos"][index] = int(df["Carga Horária"][index].split(" ")[2])
        df["Carga Horária"][index] = df["Carga Horária"][index].split(" ")[
            0].replace("h", "")
        df["Nota"][index] = float(df["Nota"][index])

        if df["Estado"][index] == "APROVADO" or "REPROVADO" in df["Estado"][index] and df["Período"][index] != "2019/2":
            cr += df["Nota"][index]*df["Créditos"][index]
            dem += df["Créditos"][index]

        try:
            df["Frequência"][index] = float(
                df["Frequência"][index].replace("%", "")) / 100
        except:
            df["Frequência"][index] = 0.0
            pass

    df = df[["Período", "Disciplina", "Professor", "Carga Horária",
             "Créditos", "Nota", "Frequência", "Estado"]]
    return df[df.Estado != "CANCELADA"], cr/dem


df = site_login(url)
print("finished")

clear_output(wait=True)

back_df = df.copy()
df, cr = process_df(back_df)
print("CR: %.2f" % cr)
#df.to_csv(r'consulta_acadêmica_Yasmin.csv', index=False)

CR: 7.97


In [3]:
df

,Período,Disciplina,Professor,Carga Horária,Créditos,Nota,Frequência,Estado
1,2016/1,CÁLCULO DIFERENCIAL E INTEGRAL I,POLYANNA POSSANI DA COSTA PETRY,90,6,7,0.9333,APROVADO
2,2016/1,GEOMETRIA ANALÍTICA,MILTON LUIZ NERI PEREIRA,60,4,5.3,0.7667,APROVADO
3,2016/1,INTRODUÇÃO À ENGENHARIA ELÉTRICA,ANDRÉ DO AMARAL PENTEADO BISCARO,60,4,8.26,0.9333,APROVADO
4,2016/1,METODOLOGIA CIENTÍFICA E REDAÇÃO CIENTÍFICA,CRISTINNE LEUS TOMÉ,60,4,8.93,0.9667,APROVADO
5,2016/1,FUNDAMENTOS DA MATEMÁTICA - NIVELAMENTO,EDSON GOMES DOS SANTOS,60,4,4,1,REPROVADO
...,...,...,...,...,...,...,...,...
68,2020/1,CONTROLADOR LÓGICO PROGRAMÁVEL,MARIA HELENA VIEIRA KELLES,60,4,NaN,0,MATRICULADO
69,2020/1,ELETIVA VI - CURSO DE ENGENHARIA ELÉTRICA,( ) EDUARDO RODRIGO THIESEN,60,4,NaN,0.0667,MATRICULADO
70,2020/1,ELETIVA VII - CURSO DE ENGENHARIA ELÉTRICA,( ) GRAZIELA ESTEVES MAGALHÃES,60,4,NaN,0.2,MATRICULADO
71,2020/1,"GERAÇÃO, TRANSMISSÃO E DISTR. DE ENERGIA ELÉTR...",NATANAEL MANOEL SILVA DOS ANJOS,75,5,NaN,0,MATRICULADO
